Alex Kappes <br>
Problem Set 6 <br>
EconS 512

In [9]:
import numpy as np
from numpy.linalg import inv
import pandas as pd

class estimate:
    def __init__(self, dep, indep, df):
        self.dep = np.asmatrix(dep).T
        self.indep = np.asmatrix(indep)
        self.df = df


    # parameters
    def beta(self):
        b = inv(self.indep.T * self.indep) * self.indep.T * self.dep
        return b


    # residuals
    def error(self, beta):
        resid = self.dep - self.indep * beta
        return resid


    # Whites HC covariance estimator
    def robust(self, e):
        k = self.indep.shape[1]
        n = self.indep.shape[0]

        s = np.zeros((k, k))
        i = 1

        while i < n:

            s = s + e[i, 0]**2 * self.indep[i].T * self.indep[i]
            i = i + 1

            if i > n:
                break

        w_hce = inv(self.indep.T * self.indep) * s * inv(self.indep.T * self.indep)
        w_se = np.sqrt(np.diag(w_hce))
        return w_se


    # construction of se clustered by state
    def se_cluster_s(self, g, e):
        group_s = pd.DataFrame(df[g].unique().tolist()).rename(columns={0: 'group'})

        l_s = {}
        for i in group_s['group']:
            l_s[i] = pd.DataFrame()

        for key in l_s:
            l_s[key] = df[df[g] == key].index.tolist()

        group_e = pd.DataFrame(e).rename(columns={0: 'ehat'})

        d_sigsq = {}
        for i in group_s['group']:
            d_sigsq[i] = pd.DataFrame()

        for key in d_sigsq:
            d_sigsq[key] = (np.asmatrix(group_e.loc[l_s[key]]).T *
                            np.asmatrix(group_e.loc[l_s[key]])) / len(l_s[key])

        sigsq_df = pd.DataFrame({'sigsq': 0}, index=df.index)

        for key in d_sigsq:
            sigsq_df.loc[df[df[g] == key].index.tolist(), 'sigsq'] = d_sigsq[key][0, 0]

        sigsq_l = np.array(sigsq_df)
        s = np.multiply(np.divide(1, sigsq_l), np.eye(len(self.indep)))
        covmat_s = inv(self.indep.T * s * self.indep)
        return np.sqrt(np.diag(covmat_s))

        # Code commented out below follows Greene's equations for clustered covmat estimator:
        # he specifies by grouping and summing over all groups but this produces singular x_g'x_g
        # matrices for all of this binary fixed effect data. I've created an identy matrix with
        # repeated 1 / sighat_g^2 values corresponding to index values in X.
        # Each within group obs is weighted appropriately and det(X) exists, providing
        # a computable clustered covmat estimator. Code below is kept for future reference
        # when not working with bullshit class data.

        # xtxi = {}
        # for state in group_s['group']:
        #     xtxi[state] = pd.DataFrame()
        #
        # for key in xtxi:
        #     xtxi[key] = inv(np.asmatrix(df[df[g] == key][x_vars]).T * np.asmatrix(df[df[g] == key][x_vars]))
        #
        # covmat_s_g = {}
        # for state in group_s['group']:
        #     covmat_s_g[state] = pd.DataFrame()
        #
        # for key in covmat_s_g:
        #     covmat_s_g[key] = d_sigsq[key] * xtxi[key]
        #
        # covmat_l = [covmat_s_g[group_s.loc[0, 'group']]]
        # i = 1
        # while i < len(group_s):
        #     covmat_l.append(covmat_s_g[group_s.loc[i, 'group']])
        #     i += 1
        #     if i > len(group_s):
        #         break
        #
        # covmat_s = sum(covmat_l)
        # return np.sqrt(np.diag(covmat_s))


    # construction of se clustered by state and year
    def se_cluster_multi(self, g1, g2, e):
        group_1 = df[g1].unique().tolist()
        group_2 = df[g2].unique().tolist()

        group_e = pd.DataFrame(e).rename(columns={0: 'ehat'})

        l_multi_idx = []
        for i in group_1:
            for j in group_2:
                l_multi_idx.append(df[(df[g1] == i) & (df[g2] == j)].index)

        i = 0
        sighat_multi_l = []
        while i < len(l_multi_idx):
            sighat_multi_l.append(np.asmatrix(group_e.loc[l_multi_idx[i].tolist()]).T *
                                  np.asmatrix(group_e.loc[l_multi_idx[i].tolist()]) / len(l_multi_idx[i]))
            i += 1
            if i > len(l_multi_idx):
                break

        sigsq_multi_df = pd.DataFrame({'sigsq_multi': 0}, index=df.index)

        for i in range(len(l_multi_idx)):
            sigsq_multi_df.loc[l_multi_idx[i].tolist()] = sighat_multi_l[i][0, 0]

        sigsq_multi_l = np.array(sigsq_multi_df)
        g_multi = np.multiply(np.divide(1, sigsq_multi_l), np.eye(len(self.indep)))
        covmat_g_multi = inv(self.indep.T * g_multi * self.indep)
        return np.sqrt(np.diag(covmat_g_multi))

**Problem 1**

In [10]:
### data construction ###
df = pd.read_csv('/home/akappes/WSU/512_MetricsII/merit_data.csv')

bin_state = pd.DataFrame(0, columns=df['state'].unique().tolist()[:-1], index=df.index)
bin_year = pd.DataFrame(0, columns=df['year'].unique().tolist()[:-1], index=df.index)

for j in bin_state.columns:
    bin_state.loc[df[df['state'] == j].index.tolist(), j] = 1

for j in bin_year.columns:
    bin_year.loc[df[df['year'] == j].index.tolist(), j] = 1

df.insert(loc=0, column='ones', value=1)
df = pd.concat([df, bin_state, bin_year], axis=1)

X = df.drop(['coll', 'year', 'state', 'chst'], axis=1)
x_vars = X.columns
y = df['coll']

**(a)**

In [1]:
est_a = estimate(y, X, df)
params_a = est_a.beta()
resids_a = est_a.error(params_a)
whites_hce_a = est_a.robust(resids_a)
clustered_state_se_a = est_a.se_cluster_s('state', resids_a)
clustered_stateyear_se_a = est_a.se_cluster_multi('state', 'year', resids_a)

print('X variables (state binaries appear in numeric categorical form): \n', x_vars.tolist())

X variables (state binaries appear in numeric categorical form): 
 ['ones', 'merit', 'male', 'black', 'asian', 11, 12, 13, 14, 15, 16, 21, 22, 23, 31, 32, 33, 34, 35, 41, 42, 43, 44, 45, 46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 71, 72, 73, 74, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 94, 1993, 1997, 1992, 1989, 1991, 1999, 1994, 1998, 2000, 1995, 1990]


In [2]:
print('Estimated parameters: \n', np.round(np.array(params_a.T), 3))

Estimated parameters: 
 [[ 0.427  0.034 -0.079 -0.15   0.169  0.033  0.08   0.037  0.173  0.115
   0.191  0.125  0.175  0.098  0.092  0.031  0.152  0.122  0.122  0.066
   0.133  0.082  0.163  0.043  0.132  0.135  0.098  0.17   0.095  0.166
   0.027  0.105  0.103  0.008  0.02  -0.001 -0.02   0.092  0.097 -0.045
   0.064  0.061  0.008  0.053 -0.014  0.038 -0.    -0.029 -0.001  0.047
  -0.063  0.042  0.04   0.059  0.004  0.002 -0.009 -0.008 -0.019 -0.01
  -0.017 -0.003  0.006 -0.007 -0.022 -0.027]]


In [3]:
print("White's HC covariance estimator standard errors: \n", np.round(whites_hce_a, 5))

White's HC covariance estimator standard errors: 
 [0.02917 0.01451 0.00476 0.00732 0.01344 0.03699 0.03843 0.03899 0.03123
 0.03867 0.03706 0.02926 0.03039 0.0302  0.03015 0.03468 0.03015 0.03011
 0.03456 0.03496 0.03471 0.03612 0.03396 0.03339 0.03409 0.03451 0.03778
 0.03813 0.03822 0.03422 0.0343  0.0307  0.03507 0.03509 0.03057 0.03501
 0.03459 0.03414 0.03495 0.03608 0.03441 0.03472 0.02963 0.03436 0.03357
 0.03481 0.03579 0.03473 0.03465 0.03334 0.03598 0.03605 0.03515 0.0285
 0.03455 0.01195 0.01229 0.01191 0.01142 0.01175 0.01222 0.01202 0.01224
 0.0125  0.01216 0.0116 ]


In [4]:
print('Clustered state standard errors: \n', np.round(clustered_state_se_a, 5))

Clustered state standard errors: 
 [0.02918 0.01449 0.00476 0.00753 0.01364 0.03699 0.03841 0.03897 0.03118
 0.03867 0.03705 0.02926 0.03042 0.0302  0.03014 0.03467 0.03015 0.03012
 0.03454 0.03493 0.03474 0.03612 0.03395 0.03337 0.0341  0.03447 0.03779
 0.03816 0.03825 0.03425 0.03428 0.03068 0.03505 0.0351  0.03055 0.03499
 0.03457 0.03412 0.03493 0.03612 0.0344  0.0347  0.02962 0.03435 0.03357
 0.03479 0.03576 0.03472 0.03464 0.03333 0.03596 0.03605 0.03516 0.02852
 0.03452 0.01197 0.01231 0.01196 0.01147 0.01178 0.01225 0.01207 0.01226
 0.01252 0.01223 0.01162]


In [5]:
print('Clustered state and year standard errors: \n', np.round(clustered_stateyear_se_a, 5))

Clustered state and year standard errors: 
 [0.02912 0.01447 0.00476 0.00752 0.01364 0.0369  0.03835 0.03889 0.03115
 0.03863 0.037   0.02922 0.03038 0.03017 0.0301  0.03462 0.03011 0.03009
 0.0345  0.03489 0.0347  0.03608 0.03392 0.03332 0.03406 0.03443 0.03769
 0.0381  0.03806 0.03421 0.03421 0.03065 0.03501 0.03499 0.03051 0.03493
 0.03444 0.03408 0.03487 0.03598 0.03434 0.03465 0.02958 0.03431 0.0335
 0.03474 0.03566 0.03462 0.03456 0.03329 0.0358  0.03599 0.03509 0.02848
 0.03447 0.01192 0.01226 0.01188 0.0114  0.01173 0.0122  0.012   0.01221
 0.01247 0.01213 0.01157]


**(b)**

In [6]:
years = df['year'].unique().tolist()

weights = []
for i in years:
    weights.append(len(df[df['year'] == i]['state'].unique()) /
                   len(df[df['year'] == i]['state']))

year_idx = []
for i in years:
    year_idx.append(df[df['year'] == i].index.tolist())

X_weighted = pd.DataFrame(0, columns=X.columns, index=X.index)

for i in range(len(weights)) and range(len(year_idx)):
    X_weighted.loc[year_idx[i]] = weights[i] * X.loc[year_idx[i]]

X_weighted['ones'] = 1

est_b = estimate(y, X_weighted, df)
params_b = est_b.beta()

print('Estimated state-weighted parameters: \n', np.round(np.array(params_b.T), 3))

Estimated state-weighted parameters: 
 [[  0.43    2.735  -5.511 -10.193  11.463   2.079   5.231   1.978  11.806
    7.884  13.033   8.218  11.893   6.689   5.991   2.082  10.207   7.991
    8.054   4.054   8.716   5.336  10.855   2.737   8.608   8.807   6.609
   11.367   5.899  11.024   1.7     6.871   6.57    0.045   0.803  -0.354
   -1.589   6.073   6.167  -3.557   3.928   3.976   0.326   3.146  -1.044
    2.287  -0.595  -2.433  -0.359   2.753  -4.46    2.489   2.384   4.069
   -0.268   0.279  -0.607  -0.38   -1.072  -0.447  -1.065  -0.083   0.364
   -0.531  -1.391  -1.738]]


**(c)**

In [7]:
s_group = df['state'].unique().tolist()
y_group = df['year'].unique().tolist()

sy_idx = []
for i in s_group:
    for j in y_group:
        sy_idx.append(df[(df['state'] == i) & (df['year'] == j)].index)

mean_l = []
i = 0
while i < len(sy_idx):
    mean_l.append(X.loc[sy_idx[i]].mean().tolist())
    i += 1
    if i > len(sy_idx):
        break

X_means = pd.DataFrame(0, columns=X.columns, index=X.index)

for i in range(len(sy_idx)) and range(len(mean_l)):
    X_means.loc[sy_idx[i]] = mean_l[i] * X.loc[sy_idx[i]]

est_c = estimate(y, X_means, df)
params_c = est_c.beta()
resids_c = est_c.error(params_c)
whites_hce_c = est_c.robust(resids_c)
clustered_state_se_c = est_c.se_cluster_s('state', resids_c)

print('Estimated (state x year) mean-weighted parameters: \n', np.round(np.array(params_c.T), 3))

Estimated (state x year) mean-weighted parameters: 
 [[ 0.337  0.034 -0.155 -0.541  0.37   0.126  0.176  0.133  0.26   0.202
   0.273  0.215  0.263  0.182  0.173  0.119  0.237  0.206  0.21   0.157
   0.223  0.163  0.252  0.133  0.218  0.223  0.195  0.271  0.421  0.265
   0.114  0.194  0.22   0.118  0.103  0.079  0.064  0.195  0.249  0.038
   0.172  0.144  0.092  0.145  0.076  0.131  0.086  0.061  0.088  0.137
   0.025  0.134  0.129  0.157  0.092  0.001 -0.008 -0.007 -0.021 -0.011
  -0.015 -0.004  0.006 -0.006 -0.021 -0.029]]


In [8]:
print("White's HC covariance estimator standard errors: \n", np.round(whites_hce_c, 5))

White's HC covariance estimator standard errors: 
 [0.05071 0.01456 0.00943 0.02919 0.07101 0.05553 0.05643 0.05685 0.05182
 0.05673 0.0556  0.05068 0.05131 0.05128 0.05123 0.05404 0.05122 0.05121
 0.0539  0.05421 0.05403 0.055   0.05356 0.05318 0.05371 0.05385 0.05598
 0.05634 0.05903 0.05357 0.05373 0.05153 0.05436 0.05432 0.05141 0.05421
 0.05397 0.05374 0.05458 0.05481 0.05388 0.05405 0.05095 0.05384 0.05335
 0.05411 0.05468 0.05397 0.05399 0.05317 0.0547  0.05489 0.05436 0.04953
 0.05381 0.01199 0.01231 0.01196 0.01146 0.0118  0.01226 0.01207 0.01227
 0.01255 0.01222 0.01164]


In [9]:
print('Clustered state standard errors: \n', np.round(clustered_state_se_c, 5))

Clustered state standard errors: 
 [0.04838 0.01453 0.00943 0.02952 0.0688  0.05341 0.05434 0.05477 0.04953
 0.05465 0.05348 0.04835 0.04904 0.04897 0.04892 0.05184 0.04892 0.0489
 0.05171 0.05203 0.05186 0.05285 0.05135 0.05095 0.05151 0.05166 0.05387
 0.05425 0.05709 0.05145 0.05153 0.04923 0.0522  0.05218 0.04912 0.05204
 0.05177 0.05154 0.05244 0.05271 0.05169 0.05186 0.04861 0.05164 0.05112
 0.05192 0.05251 0.05179 0.05179 0.05094 0.05256 0.05274 0.05218 0.04722
 0.05162 0.012   0.01234 0.012   0.01151 0.01182 0.01228 0.01211 0.01229
 0.01256 0.01226 0.01165]


**(d)**

In [15]:
weights_sample = []
i = 0
while i < len(sy_idx):
    weights_sample.append(len(sy_idx[i]) / len(df))
    i += 1
    if i > len(sy_idx):
        break

X_weighted_sample = pd.DataFrame(0, columns=X.columns, index=X.index)

for i in range(len(sy_idx)) and range(len(weights_sample)):
    X_weighted_sample.loc[sy_idx[i]] = weights_sample[i] * X.loc[sy_idx[i]]

X_weighted_sample['ones'] = 1

est_d = estimate(y, X_weighted_sample, df)
params_d = est_d.beta()
resids_d = est_d.error(params_d)
whites_hce_d = est_d.robust(resids_d)
clustered_state_se_d = est_d.se_cluster_s('state', resids_d)

print('Estimated (state x year) sample-weighted parameters: \n', np.round(np.array(params_d.T), 3))

Estimated (state x year) sample-weighted parameters: 
 [[   0.484    3.356  -15.132  -36.136   36.517  -87.808  -23.162  -89.811
    34.389    3.583   88.638   17.508   32.135    8.558    9.378  -62.298
    24.237   16.746   19.353  -25.441   35.111  -20.469   54.861  -27.737
    28.511   32.967  -36.832   56.776 -141.059   43.441  -59.926   12.129
   -15.981  -73.232  -10.983  -90.609 -121.303  -21.949  -24.387 -109.354
   -45.007  -39.801   -9.853  -30.251  -74.864  -52.429 -105.51  -100.644
   -83.292  -32.756 -182.685  -60.21   -51.361    3.885  -62.669   -3.492
     1.672   -6.57    -6.044   -6.356   -2.551   -4.737    0.212    0.668
    -8.868   -8.883]]


In [12]:
print("White's HC covariance estimator standard errors: \n", np.round(whites_hce_d, 5))

White's HC covariance estimator standard errors: 
 [1.356000e-02 6.265690e+00 1.411750e+00 2.182240e+00 2.905990e+00
 3.321189e+01 4.090009e+01 4.482336e+01 7.520940e+00 4.420736e+01
 3.380582e+01 4.615000e+00 6.930440e+00 6.254070e+00 5.817680e+00
 2.200947e+01 6.229220e+00 6.102910e+00 1.989302e+01 2.249561e+01
 2.209227e+01 2.879429e+01 1.845291e+01 1.596627e+01 1.923280e+01
 2.063663e+01 4.118049e+01 4.121274e+01 4.562271e+01 1.961747e+01
 1.991862e+01 6.783470e+00 2.117533e+01 1.922888e+01 6.456220e+00
 2.438242e+01 2.301748e+01 1.894200e+01 1.988481e+01 2.182931e+01
 2.004644e+01 2.233140e+01 5.180790e+00 1.968578e+01 1.779068e+01
 2.167989e+01 2.855875e+01 2.187091e+01 2.092048e+01 1.636134e+01
 2.957726e+01 2.892542e+01 2.390174e+01 4.036320e+00 2.054738e+01
 4.051590e+00 4.401220e+00 3.904200e+00 3.749060e+00 3.737910e+00
 4.221460e+00 4.011250e+00 4.272170e+00 4.364230e+00 4.225840e+00
 3.777480e+00]


In [13]:
print('Clustered state standard errors: \n', np.round(clustered_state_se_d, 5))

Clustered state standard errors: 
 [1.355000e-02 6.245980e+00 1.412030e+00 2.198860e+00 2.962930e+00
 3.356672e+01 4.092342e+01 4.477380e+01 7.520570e+00 4.422291e+01
 3.373421e+01 4.620380e+00 6.941530e+00 6.259820e+00 5.822170e+00
 2.210912e+01 6.240630e+00 6.113860e+00 1.989127e+01 2.250216e+01
 2.209887e+01 2.877686e+01 1.845523e+01 1.593987e+01 1.923494e+01
 2.063284e+01 4.113209e+01 4.123811e+01 4.552253e+01 1.963529e+01
 2.000581e+01 6.781370e+00 2.124236e+01 1.915980e+01 6.469110e+00
 2.436946e+01 2.312338e+01 1.893482e+01 1.989413e+01 2.175095e+01
 2.004742e+01 2.238446e+01 5.193260e+00 1.969674e+01 1.779259e+01
 2.177204e+01 2.855009e+01 2.190453e+01 2.105134e+01 1.644263e+01
 2.990321e+01 2.906689e+01 2.408817e+01 4.053680e+00 2.033916e+01
 4.065990e+00 4.402910e+00 3.945900e+00 3.771500e+00 3.759640e+00
 4.243240e+00 4.034350e+00 4.290890e+00 4.358280e+00 4.247350e+00
 3.791780e+00]


The above estimation results show that state-weighted and sample-weighted data produce estimates with higher magnitudes. I'm not sure why we are doing OLS regression on a binary dependent variable though. The estimates result in non-marginal shift effects for if the individual went to college (assuming that's what $coll = 1$ means). The standard errors for state-weighted and sample weighted data producer greater variability than standard errors from mean-weighted and non-weighted data.

**(e)** First, the merit states are found.

In [11]:
merit_states = df[df['merit'] == 1]['state'].unique().tolist()
print('Merit states are:', merit_states)

Merit states are: [34, 57, 58, 59, 61, 64, 71, 72, 85, 88]


For the corresponding states, the first year in which merit was received is found.

In [12]:
yr_implemented = []
for i in merit_states:
    yr_implemented.append(df[(df['merit'] == 1) & (df['state'] == i)]['year'].min())

merit_states_beginning = pd.DataFrame({'year': yr_implemented}, index=merit_states)

The corresponding beginning merit years are:

In [13]:
merit_states_beginning

,year
34,2000
57,1998
58,1993
59,1997
61,1999
64,1996
71,1991
72,1998
85,1997
88,2000


Effects of merit are found for each state by differencing mean $coll$ for the year when merit was received with mean $coll$ from the year prior when merit had not yet been received. Aggregation of all merit-state differences will provide an asymptotically unbiased treatment effect.

In [14]:
def effect(m):

    merit_states = df[df['merit'] == 1]['state'].unique().tolist()
    yr = []
    for i in merit_states:
        if m == 1:
            yr.append(df[(df['merit'] == m) & (df['state'] == i)]['year'].min())
        else:
            yr.append(df[(df['merit'] == m) & (df['state'] == i)]['year'].max())
    
    merit_states_df = pd.DataFrame({'year': yr}, index=merit_states)

    effects = []
    for i in merit_states_df.index:
        effects.append(df[(df['merit'] == m) &
                          (df['state'] == i) &
                          (df['year'] == merit_states_df.loc[i, 'year'])]['coll'].mean())

    return effects

effects_df = pd.DataFrame({'prior mean coll': np.round(effect(0), 4),
                           'post mean coll': np.round(effect(1), 4)},
                          index=merit_states)

The mean effects for post and prior merit are presented below.

In [15]:
effects_df

,prior mean coll,post mean coll
34,0.5154,0.4724
57,0.4857,0.5405
58,0.3556,0.3793
59,0.3361,0.4567
61,0.4390,0.4211
64,0.3659,0.4773
71,0.2708,0.4000
72,0.3651,0.3889
85,0.3333,0.3200
88,0.3333,0.4074


Evaluating state-level differences show some states experience negative effects on $coll$ after receiving the merit.

In [27]:
effects_df['diff'] = effects_df['post mean coll'] - effects_df['prior mean coll']
effects_df

,prior mean coll,post mean coll,diff
34,0.5154,0.4724,-0.0430
57,0.4857,0.5405,0.0548
58,0.3556,0.3793,0.0237
59,0.3361,0.4567,0.1206
61,0.4390,0.4211,-0.0179
64,0.3659,0.4773,0.1114
71,0.2708,0.4000,0.1292
72,0.3651,0.3889,0.0238
85,0.3333,0.3200,-0.0133
88,0.3333,0.4074,0.0741


The merit treatment effect is approximated as

In [30]:
round(sum(effects_df['diff']), 3)

0.463

**Problem 2**

**(a)**